In [1]:
import sys
import os
import numpy as np
import pandas as pd
from typing import  Optional, Callable, Dict
from collections.abc import Mapping
import re, unicodedata


sys.path.append(os.path.abspath('..'))
from core.s3 import S3AssetManager

from core.viz import (
        plot_gauge_grid,
        plot_bar,
        plot_heatmap,
        plot_pie,
        plot_time_heatmap
)

In [2]:
notebook_name = "okuo_production"
s3 = S3AssetManager(notebook_name=notebook_name)

## RECATEGORIZATION

In [3]:
rename_paros = {
    # --- PRIORIDAD 1: SIN PRODUCTO (Logística/Planeación) ---
    'sin producto': 'Sin Producto',
    'sin prducto': 'Sin Producto',
    'sin producto fallo de bandas de transportador': 'Sin Producto', # Prioridad aplicada
    'sin producto y atorado de maquina': 'Sin Producto', # Prioridad aplicada
    'cambio de dado, sin producto': 'Sin Producto',
    'sin prod,cambio de dado ,saranda': 'Sin Producto',
    'limpieza de maquina y sin producto': 'Sin Producto',
    'sin producto  y tolva llena': 'Sin Producto',
    'sin producto y tolva llena': 'Sin Producto',
    'cambio de dado  ,sin producto': 'Sin Producto',
    'cambio de dado ,sin producto': 'Sin Producto',
    'sin prod  cambio de dado': 'Sin Producto',
    'Sin prod y cambio de dado ': 'Sin Producto',
    'falla del sistema /sin producto': 'Sin Producto',
    'sin producto falla dosificador de balanza': 'Sin Producto',

    # --- CAMBIO DE HERRAMENTAL (Dado/Rodillo/Saranda) ---
    'cambio de dado y saranda': 'Cambio Dado/Rodillo',
    'cambio de dado': 'Cambio Dado/Rodillo',
    'cambio de dado ': 'Cambio Dado/Rodillo',
    'Ajuste de rodillos': 'Cambio Dado/Rodillo',
    'cambio de dado,rodillos,saranda': 'Cambio Dado/Rodillo',
    'cambio de rodillos': 'Cambio Dado/Rodillo',
    'cambio de rodillo': 'Cambio Dado/Rodillo',
    'cambio de dado a nuevo /rodillos ': 'Cambio Dado/Rodillo',
    'cambio de ejes ,enrgrasado': 'Cambio Dado/Rodillo', # Mantenimiento operativo de dados/rodillos

    # --- TOLVA LLENA (Cuellos de botella) ---
    'tolva llena': 'Tolva Llena',
    'Tolva llena': 'Tolva Llena',
    'tolva llena ': 'Tolva Llena',
    'tolva granel llena ': 'Tolva Llena',
    'transp granel ocupado': 'Tolva Llena',
    'transportador ocupado': 'Tolva Llena',
    'tolva llena ,falla crambell': 'Tolva Llena',

    # --- ATASQUES (Flujo detenido) ---
    'atorado de maquina': 'Atasque',
    'atorado de maquina ': 'Atasque',
    'atorado de maq': 'Atasque',
    'atorado de maquima y acond ': 'Atasque',
    'suspendido  no pasa el producto ': 'Atasque',
    'producto no pasa ': 'Atasque',
    'producto no pasa/cambio de dado': 'Atasque',
    'atorado de cambio de tolva a granel': 'Atasque',
    'atorado de maquina/no pasa E1  ': 'Atasque',

    # --- LIMPIEZA ---
    'limpieza de peletizadora ': 'Limpieza',
    'limpieza pellet ': 'Limpieza',
    'ensacado de desinfeccion': 'Limpieza',

    # --- FALLA TRANSPORTE (Elevadores/Cintas rotas) ---
    'daño de elevador de producto terminado': 'Falla del Sistema',
    'falla del elevador ': 'Falla del Sistema',
    'falla tolva b3  puerta de apertura ': 'Falla del Sistema',

    # --- MANTENIMIENTO (Mecánico General/Varios) ---
    'mantenimiento de maquinA': 'Mantenimiento',
    'sensor de peller dañado': 'Mantenimiento',
    'mant motores ': 'Mantenimiento',
    'falla de cosedora': 'Mantenimiento',
    'tolva llena reparacion de banda ': 'Mantenimiento', # Es reparación (prioridad sobre tolva llena)
    'ensacado de producto en pelet/harina': 'Mantenimiento',
    'instalacion pelet3': 'Mantenimiento', # Instalación/Upgrade
    '*': 'Mantenimiento',

    # --- FALLA CALDERO / VAPOR ---
    'falla del Calderó': 'Falla Caldero',
    'falla del caldero mantenimiento ': 'Falla Caldero',
    'Falla de caldero': 'Falla Caldero',
    'falla de caldero': 'Falla Caldero',
    'falla de caldero ': 'Falla Caldero',
    'falla de cadero': 'Falla Caldero', # Typo corregido
    'mant. caldero': 'Falla Caldero',
    'sin vapor': 'Falla Caldero',
    'sin vapor ': 'Falla Caldero',

    # --- FALLA SISTEMA ---
    'Falla del sistema': 'Falla del Sistema',
    'falla del sistema': 'Falla del Sistema',
    'falla del sistema ': 'Falla del Sistema',

    # --- FALLA ELECTRICA ---
    'sin luz': 'Falla Electrica',
    'sin luz ': 'Falla Electrica',
    'falla pellet fusibles rotos': 'Falla Electrica',

    # --- FALLA VENTILADOR ---
    'cambio bandas ventilador': 'Falla del Sistema',
    'falla  ventilador ': 'Falla del Sistema',

    # --- EXPERIMENTAL / CALIDAD ---
    'EXP T1': 'Experimental',
    'EXP T2': 'Experimental',
    'EXP T3': 'Experimental',
    'prod por comfirmar ': 'Experimental' # Pausa por validación de calidad
}
map_tipo_actividad = {
    # --- IMPRODUCTIVO ---
    "Sin Producto": "Improductivo",
    "Atasque": "Improductivo",
    "Tolva Llena": "Improductivo",
    "Falla Caldero": "Improductivo",   # Se considera falta de suministro (vapor)
    "Falla del Sistema": "Improductivo",
    "Falla Electrica": "Improductivo",   # Se considera falta de suministro (energía)

    # --- MANTENIMIENTO ---
    "Cambio Dado/Rodillo": "Mantenimiento",
    "Limpieza": "Mantenimiento",
    "Mantenimiento": "Mantenimiento",
    "Falla Ventilador": "Mantenimiento", # Falla mecánica de componente interno

    # --- AGENTE EXTERNO ---
    "Falla Transporte": "Agente Externo",

    # --- CALIDAD ---
    "Experimental": "Calidad"
}

alimento_a_especie = {
    "G5PR": "ganaderia",
    "F-2 PREMEX": "cerdos",
    "F-3 PREMEX": "cerdos",
    "F-3": "cerdos",
    "F-4": "cerdos",
    "F-5": "cerdos",
    "F4 PREMEX": "cerdos",
    "F5": "cerdos",
    "F5 PREMEX": "cerdos",
    "F1 PREMEX": "cerdos",
    "F2 PREMEX": "cerdos",
    "F3 PREMEX": "cerdos",
    "P-2": "cerdos",
    "P-3": "cerdos",
    "P-5": "cerdos",
    "P2": "cerdos",
    "P3": "cerdos",
    "P4": "cerdos",
    "P5": "cerdos",
    "P6": "cerdos",
    "C-1": "cerdos",
    "CE1": "cerdos",
    "CE-1": "cerdos",
    "CR-2": "cerdos",
    "CR2": "cerdos",
    "CF": "cerdos",
    "CG1": "cerdos",
    "AIP": "ponedora",
    "AIP -PREMEX": "ponedora",
    "APP": "ponedora",
    "BAF": "ponedora",
    "BAI": "ponedora",
    "BCE": "cerdos",
    "BCC": "cerdos",
    "BCG": "cerdos",
    "BCL": "cerdos",
    "ACP": "ponedora",
    'C-2': 'cuyes',
    'F1':"cerdos",
    'CE2':"cerdos",
    'F2':"cerdos",
    'F3':"cerdos",
    'CL':"cerdos",
    "CR1":"cerdos",
    'F4':"cerdos",

    'C1': 'cuyes',
    'F-1 PREMEX':"cerdos",
    'F-4 PREMEX':"cerdos",
    'P-4': 'cerdos',

}

def clasificar_dieta(d):
    if pd.isna(d):
        return np.nan

    s = str(d).strip().upper()
    if s == "" or "*" in s:
        return np.nan
    if s in alimento_a_especie:
        return alimento_a_especie[s]
    token = s.split()[0]

    # 2) reglas por prefijo
    if token.startswith("HY"):
        return "reproductoras"
    if token.startswith("E"):
        return "broiler"
    if token.startswith("PF"):
        return "cerdos"
    if token.startswith("PCR"):
        return "maquila"
    if token.startswith("G"):
        return "ganaderia"
    return "desconocido"



def canonize(col: str) -> str:
    s = col.strip().replace('\n', ' ')
    s = re.sub(r'\.\d+$', '', s)
    s = unicodedata.normalize('NFKD', s)
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    s = s.lower()
    s = re.sub(r'[^a-z0-9]+', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

ES2EN = {
    'fecha': 'date',
    'hora inicio': 'start_time',
    'tipo de alimento': 'feed_type',
    'lote': 'lot',
    'cant tm': 'quantity_tm',
    'temp c': 'temperature_c',
    'tiempo de para': 'downtime',
    'observaciones': 'notes',
    'hora final': 'end_time',
    'tiempo acumulado': 'accumulated_time',
    'operador': 'operator',
}


FINAL_ORDER = [
    'date','start_time','end_time','accumulated_time','downtime',
    'feed_type','lot','quantity_tm','temperature_c','operator','notes','pellet'
]

def standardize(df: pd.DataFrame, pellet_label: str) -> pd.DataFrame:
    out = df.copy()
    out['pellet'] = pellet_label
    out = out.rename(columns=lambda c: ES2EN.get(canonize(c), canonize(c)))
    cols = [c for c in FINAL_ORDER if c in out.columns]
    return out[cols]


def limpiar_fecha(s: str) -> str | None:
    if pd.isna(s):
        return None
    s = str(s).strip()
    if not s:
        return None

    # Reemplazar ':' por '/' en cosas como '30/09:2025'
    s = s.replace(":", "/")
    # Arreglar cosas tipo '30/092025' -> '30/09/2025'
    m = re.match(r"^(\d{1,2})/(\d{2})(\d{4})$", s)
    if m:
        s = f"{m.group(1)}/{m.group(2)}/{m.group(3)}"
    return s

In [4]:
def resumen_cumplimiento_rango(
    df,
    specie_col: str,
    lot_col: str,
    value_col: str,
    target_min,
    target_max,
    unit: str = "",
    dropna: bool = False,
):
    """
    Calcula cumplimiento de rango por especie para un valor (humedad, temperatura, etc.)

    Parámetros
    ----------
    df : DataFrame
    specie_col : str
        Nombre de la columna con la especie (por ejemplo 'specie').
    lot_col : str
        Nombre de la columna de lote (por ejemplo 'lot').
    value_col : str
        Nombre de la columna con el valor a evaluar (por ejemplo 'hum').
    target_min :
        Puede ser:
          - número (ej. 11.5)
          - dict {especie: min}
          - nombre de columna con el mínimo
    target_max :
        Puede ser:
          - número (ej. 12.5)
          - dict {especie: max}
          - nombre de columna con el máximo
    unit : str
        Unidad para mostrar en el rango_objetivo (ej. "%", "°C").
    dropna : bool
        Pasar a groupby(dropna=dropna). Útil si quieres ver especies NaN.

    Retorna
    -------
    DataFrame con columnas:
      specie, rango_objetivo, valor_promedio,
      cantidad_lotes, lotes_en_rango, pct_cumplimiento
    """

    dg = df.copy()

    # --- construir columnas min y max en función de los argumentos ---
    def build_target_column(source, col_name):
        # source puede ser número, dict o nombre de columna
        if isinstance(source, Mapping):
            # dict -> map por especie
            return dg[specie_col].map(source)
        elif isinstance(source, (int, float, np.floating)):
            # valor constante
            return float(source)
        elif isinstance(source, str):
            # nombre de columna ya existente
            return dg[source]
        else:
            raise ValueError(f"No sé cómo interpretar target para {col_name}: {source}")

    dg["__min"] = build_target_column(target_min, "min")
    dg["__max"] = build_target_column(target_max, "max")

    # 1) Promedio por lote y especie
    ops = (
        dg
        .groupby([specie_col, lot_col], as_index=False, dropna=dropna)
        .agg(
            valor_min=("__min", "first"),
            valor_max=("__max", "first"),
            valor_promedio=(value_col, "mean"),
        )
    )

    # 2) Lotes dentro del rango
    ops["in_range"] = ops["valor_promedio"].between(
        ops["valor_min"],
        ops["valor_max"],
        inclusive="both"
    )

    # 3) Resumen por especie
    summary = (
        ops
        .groupby(specie_col, dropna=dropna)
        .agg(
            valor_min=("valor_min", "first"),
            valor_max=("valor_max", "first"),
            valor_promedio=("valor_promedio", "mean"),
            cantidad_lotes=(lot_col, "nunique"),
            lotes_en_rango=("in_range", "sum"),
        )
        .reset_index()
    )

    # 4) Porcentaje de cumplimiento
    summary["pct_cumplimiento"] = (
        summary["lotes_en_rango"] / summary["cantidad_lotes"] * 100
    )

    # 5) Columna de texto con rango objetivo
    label_unit = f" {unit}" if unit else ""
    summary["rango_objetivo"] = (
        "("
        + summary["valor_min"].round(1).astype(str)
        + " - "
        + summary["valor_max"].round(1).astype(str)
        + ")"
        + label_unit
    )

    # 6) Ordenar columnas y redondear
    summary = summary[
        [
            specie_col,
            "rango_objetivo",
            "valor_promedio",
            "cantidad_lotes",
            "lotes_en_rango",
            "pct_cumplimiento",
        ]
    ]

    summary = summary.round(
        {
            "valor_promedio": 2,
            "pct_cumplimiento": 2,
        }
    ).sort_values("pct_cumplimiento", ascending=True)
    summary = summary.rename(columns={
        'specie': "Especie",
        'rango_objetivo': "Rango objetivo",
        'valor_promedio': "Valor promedio",
        'cantidad_lotes': "Cantidad de lotes",
        'lotes_en_rango': "Lotes en rango",
        'pct_cumplimiento': "Pct cumplimiento",

    })

    return summary


In [5]:
def _to_timedelta_robusto(x):
    """Convierte valores tipo '45min', '45 min', '45 minutos', 'HH:MM:SS', Timedelta, etc. a pd.Timedelta."""
    if pd.isna(x):
        return pd.NaT
    if isinstance(x, (pd.Timedelta, np.timedelta64)):
        return pd.to_timedelta(x)

    s = str(x).strip().lower()
    if s in {"", "nan", "nat", "none"}:
        return pd.NaT

    # Casos tipo "45min" / "45 min" / "45 minutos"
    m = re.fullmatch(r"(\d+(?:\.\d+)?)\s*(min|mins|min\.|minuto|minutos)$", s)
    if m:
        return pd.to_timedelta(float(m.group(1)), unit="m")

    # Casos tipo "2h" / "2 horas"
    h = re.fullmatch(r"(\d+(?:\.\d+)?)\s*(h|hr|hrs|hora|horas)$", s)
    if h:
        return pd.to_timedelta(float(h.group(1)), unit="h")

    # Intento general: "HH:MM:SS", "0 days 00:45:00", etc.
    return pd.to_timedelta(s, errors="coerce")

def join_unique(x):
    if isinstance(x, (list, tuple, set, np.ndarray, pd.Series)):
        vals = pd.Series(x).dropna().astype(str)
        vals = vals[vals.str.strip() != ""]
        return ", ".join(sorted(vals.unique()))
    return str(x)

In [6]:


# --- Helper de Tiempo (Reutilizable) ---
def _hours_to_hms_str(hours: float) -> str:
    """Convierte float de horas a 'HH:MM:SS'."""
    if hours is None or pd.isna(hours):
        return "00:00:00"
    seconds = int(round(float(hours) * 3600))
    seconds = max(seconds, 0)
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return f"{h:02d}:{m:02d}:{s:02d}"


def prepare_pie_data(
    df: pd.DataFrame,
    category_col: str,
    value_col: Optional[str] = None,  # Opcional si solo quieres contar eventos (frecuencia)

    # Configuración de qué métricas incluir en el cálculo y hover
    include_sum: bool = True,
    include_mean: bool = True,
    include_count: bool = True,

    # Personalización del Hover (Etiquetas y Formateadores)
    labels: Dict[str, str] = None,    # Ej: {"sum": "Tiempo Total", "count": "N eventos"}
    formatters: Dict[str, Callable] = None # Ej: {"mean": _hours_to_hms_str}
) -> pd.DataFrame:
    """
    Procesa los datos crudos, agrupa por categoría y construye un string HTML
    para el hover con las métricas deseadas.

    Args:
        labels: Diccionario para renombrar métricas en el hover ('sum', 'mean', 'count').
        formatters: Diccionario de funciones para dar formato a los números.
    Returns:
        pd.DataFrame: DataFrame con columnas ['label', 'slice_value', 'hover_html'] listo para graficar.
    """
    d = df.copy()

    # 1. Limpieza básica
    d[category_col] = d[category_col].fillna("No especificado").astype(str).str.strip()
    d = d[d[category_col] != ""]

    # 2. Definir Agregaciones
    agg_funcs = {category_col: "size"} # Siempre calculamos count ('size')

    if value_col:
        # Asegurar numérico
        d[value_col] = pd.to_numeric(d[value_col], errors="coerce").fillna(0)
        agg_funcs[value_col] = ["sum", "mean"]

    # 3. Agrupación
    grouped = d.groupby(category_col).agg(agg_funcs)

    # 4. Aplanar MultiIndex y estandarizar nombres de columnas
    if value_col:
        # Resultado ej: category_col_size, value_col_sum, value_col_mean
        grouped.columns = ["_".join(col).strip() if col[1] else col[0] for col in grouped.columns.values]

        # Mapeo a nombres estándar internos
        col_map = {
            f"{category_col}_size": "count",
            f"{value_col}_sum": "sum",
            f"{value_col}_mean": "mean"
        }
    else:
        grouped.columns = ["count"]
        col_map = {}

    grouped = grouped.rename(columns=col_map).reset_index()

    # 5. Definir el valor principal de la rebanada (slice_value)
    # Si hay value_col y pedimos suma, la torta es por suma. Si no, es por conteo.
    main_metric = "sum" if (value_col and include_sum) else "count"
    grouped["slice_value"] = grouped[main_metric]
    grouped = grouped.sort_values("slice_value", ascending=False)

    # 6. CONSTRUCCIÓN DINÁMICA DEL HOVER HTML
    # Valores por defecto
    default_labels = {"sum": "Total", "mean": "Promedio", "count": "Eventos"}
    default_labels.update(labels or {})

    default_formatters = {
        "sum": lambda x: f"{x:.2f}",
        "mean": lambda x: f"{x:.2f}",
        "count": lambda x: f"{int(x)}"
    }
    default_formatters.update(formatters or {})

    def build_hover(row):
        # Título del tooltip (la categoría)
        lines = [f"<b>{row[category_col]}</b>"]

        # Línea de Suma (si aplica)
        if value_col and include_sum:
            val_str = default_formatters["sum"](row["sum"])
            lines.append(f"{default_labels['sum']}: {val_str}")

        # Línea de Promedio (si aplica)
        if value_col and include_mean:
            val_str = default_formatters["mean"](row["mean"])
            lines.append(f"{default_labels['mean']}: {val_str}")

        # Línea de Conteo (si aplica)
        if include_count:
            val_str = default_formatters["count"](row["count"])
            lines.append(f"{default_labels['count']}: {val_str}")

        return "<br>".join(lines)

    grouped["hover_html"] = grouped.apply(build_hover, axis=1)

    # Retornamos el nombre de la categoría en una columna estándar
    grouped = grouped.rename(columns={category_col: "label"})

    return grouped

In [7]:
# base = ../raw/
path_base = "raw/aliforte"
qa_base = s3.read_excel(
    f"{path_base}/AGOSTO HUM Y PDI.xlsx",
    sheet_name='2025',
    skiprows=4,
)
qa_sep = s3.read_excel(f"{path_base}/HUM PDI SEPTIEMBRE 2025 (1).xlsx",skiprows=4, sheet_name='2025',)
qa_oct = s3.read_excel(f"{path_base}/PDI - HUM OCT 2025 (1).xlsx",skiprows=4,  sheet_name='2025',)
qa_nov = s3.read_excel(f"{path_base}/PDI y HUMEDAD NOV 2025.xlsx",skiprows=4,  sheet_name='2025',)


prod_base = s3.read_excel(
    f"{path_base}/PRODUCTIVIDAD DE LAS PLLET.xlsx",
    sheet_name='DATOS',
    skiprows=3,
    )

prod_new = s3.read_excel(
    f"{path_base}/Procutividad Pellet_filter.xlsx",
    sheet_name='Pellet ',
    skiprows=1,
)

prod_nov  = s3.read_excel(
    f"{path_base}/Procutividad Pellet.xlsx",
    sheet_name='DATOS',
    skiprows=1,
)

In [8]:
qa = pd.concat([ qa_base,qa_sep, qa_oct, qa_nov])
rename_qa = {
    "LOTE":"lot",
    'TIPO DE PRODUCTO': 'product',
    "HUMEDAD MÁX 13%": 'hum',
    '> 85 PDI (%)': 'pdi'
}
qa_dep = qa[rename_qa.keys()].copy()
qa_dep = qa_dep.rename(columns=rename_qa)

col_qa_num = ["lot", "pdi", "hum"]
for cl_qa in col_qa_num:
    qa_dep[cl_qa] = pd.to_numeric(qa_dep[cl_qa], errors='coerce')
qa = qa_dep[qa_dep["lot"].notnull()][["lot", "product", "hum", "pdi"]]

prod = pd.concat([ prod_base,prod_new,prod_nov])
prod_pel1_cols = [
'FECHA', 'HORA INICIO', 'TIPO DE ALIMENTO', 'LOTE', 'CANT. TM',
'TEMP. °C', 'TIEMPO DE PARA', 'OBSERVACIONES', 'HORA FINAL',
'TIEMPO \nACUMULADO', 'OPERADOR',
]
prod_pel1 = prod[prod_pel1_cols].copy()
prod_pel1.loc[:, "pellet"] ="pelle 1"

prod_pel2_cols = [
'FECHA.1',
'HORA INICIO.1', 'TIPO DE ALIMENTO.1', 'LOTE.1', 'CANT. TM.1',
'TEMP. °C.1', 'TIEMPO DE PARA.1', 'OBSERVACIONES.1', 'HORA FINAL.1',
'TIEMPO \nACUMULADO.1', 'OPERADOR.1',
]
prod_pel2 = prod[prod_pel2_cols].copy()
prod_pel2.loc[:, "pellet"] ="pelle 2"

prod_pel3_cols = [
'FECHA.2',
'HORA INICIO.2', 'TIPO DE ALIMENTO.2', 'LOTE.2', 'CANT. TM.2',
'TEMP. °C.2', 'TIEMPO DE PARA.2', 'OBSERVACIONES.2', 'HORA FINAL.2',
'TIEMPO \nACUMULADO.2', 'OPERADOR.2'
]
prod_pel3 = prod[prod_pel3_cols].copy()
prod_pel3.loc[:, "pellet"] ="pelle 3"

prod_pel1_std = standardize(prod_pel1, 'pellet 1')
prod_pel2_std = standardize(prod_pel2, 'pellet 2')
prod_pel3_std = standardize(prod_pel3, 'pellet 3')


prod_pel = pd.concat([prod_pel1_std, prod_pel2_std, prod_pel3_std], ignore_index=True)
prod_pel['date'] = pd.to_datetime(prod_pel['date'],  dayfirst=True,errors='coerce')
prod_pel['month'] = prod_pel['date'].dt.month
prod_pel = prod_pel[prod_pel['date'].notnull()].copy()

prod_pel["accumulated_td"] = prod_pel["accumulated_time"].apply(_to_timedelta_robusto)
prod_pel["hour"] = prod_pel["accumulated_td"].dt.total_seconds() / 3600

prod_pel["downtime_str"] = prod_pel["downtime"].astype(str).replace({'45min': '00:45:00'})

prod_pel["downtime_td"] = pd.to_timedelta(
    prod_pel["downtime_str"].astype(str),
)
prod_pel["downtime_hour"] = prod_pel["downtime_td"].dt.total_seconds() / 3600

prod_pel["temperature_c"] = pd.to_numeric(prod_pel['temperature_c'], errors='coerce')
prod_pel["quantity_tm"] = pd.to_numeric(prod_pel['quantity_tm'], errors='coerce')

prod_pel["start_hour"] = prod_pel['date'].astype(str) + ' ' +prod_pel['start_time'].astype(str)
prod_pel["start_hour"] = pd.to_datetime(prod_pel['start_hour'], errors='coerce')
prod_pel["ts_round"] = prod_pel["start_hour"].dt.round("H")

prod_pel["normalize_notes"] = prod_pel["notes"].replace(rename_paros)
prod_pel["details_notes"] = prod_pel["normalize_notes"].map(map_tipo_actividad)
prod_pel["details_notes"] = prod_pel["details_notes"].fillna("Productivo")

prod_pel["operator"] = (
    prod_pel["operator"]
        .astype("string")
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.title()
)
prod_pel["operator"]= prod_pel["operator"].replace({
    'Jose Benacazar': 'Jose Benalcazar',
})
prod_pel["feed_type"] = prod_pel["feed_type"].str.upper()
cond_bad = prod_pel["feed_type"].isin(['*',  ' ', 'MINUTOS', np.nan])

prod_pel["feed_type"] = prod_pel["feed_type"].replace({
    "E1": "E-1", "E2": "E-2", "E3": "E-3", "E4": "E-4",
    #"G1": "G-1", "G2": "G-2", "G3": "G-3", 'G 1': "G-1",
    #'C1': "C-1", "F1": "F-1", "F2": "F-2", "F3": "F-3", 'F4': "F-4",
    #"P1": "P-1", "P2": "P-2", "P3": "P-3", 'P4': "P-4",

})
prod_pel["feed_type"] = np.where(~cond_bad, prod_pel["feed_type"], '')
prod_pel["feed_type"]  = prod_pel["feed_type"].str.strip()

prod_pel["specie"] = prod_pel["feed_type"].apply(clasificar_dieta)

prod_pel["lot"] = pd.to_numeric(prod_pel["lot"], errors="coerce")
ops_pel = prod_pel[prod_pel["lot"].notnull()].copy()
sin_ops_pel = prod_pel[prod_pel["lot"].isnull()].copy()

In [9]:

prod_pel_group = ops_pel.groupby(["lot", "pellet"], dropna=False).agg(
date=('date', 'first'),
feed_type=('feed_type', 'first'),
specie=('specie', 'first'),
start_time=('start_time', 'first'),
quantity_tm=('quantity_tm', 'sum'),
hour=('hour', 'sum'),
temperature_c=('temperature_c', 'mean'),
rows=('pellet', 'count'),
downtime_hour=('downtime_hour', 'sum'),
notes=('normalize_notes', 'first'), # prima el primer motivo de paro
details_notes=('details_notes', 'first'),
operators=('operator', 'first') # lotes compartidos por tuno, prima el operario incial
).reset_index()
prod_pel_group["month"] = prod_pel_group["date"].dt.month
prod_pel_group["performance"] = prod_pel_group["quantity_tm"]/prod_pel_group["hour"]
prod_pel_group["with_notes"] = np.where(prod_pel_group["notes"].isnull(), "sin paro", "con paro")

# TODO A cada op se le asignan las medidas de calidad
prod_pel_group  = pd.merge(
   prod_pel_group,
    qa, on='lot',
    how='left')

prod_pel_group["performance"] = pd.to_numeric(prod_pel_group["performance"], errors='coerce')

#TODO: producción, evitar malas marcaciones
prod_pel_group["performance"] = prod_pel_group["performance"].replace([np.inf, -np.inf], np.nan)
prod_pel_group = prod_pel_group.sort_values(["lot", "date"]).copy()
prod_pel_group["lot"] = prod_pel_group["lot"].astype(int)

# Todo: tirajes cortos menores o iguales a 15 ton
cond_tiro = prod_pel_group["quantity_tm"] <= 15
prod_pel_group["tiro"] = np.where(cond_tiro, "corto", "largo")

In [10]:
# TODO: pellet 2 con mayor densidad de lotes cera a 0Ton/h


quantiles_map = {
    "pellet 1": (0.03, 0.99),
    "pellet 3": (0.03, 0.99),
    "pellet 2": (0.07, 0.99),
}
default_q = (0.03, 0.99)


df_base = prod_pel_group.copy()
df_pos = df_base[df_base["performance"] > 0].copy()

q_by_pellet = (
    df_pos[["pellet"]]
    .drop_duplicates()
    .assign(
        q_low=lambda d: d["pellet"].map(lambda x: quantiles_map.get(x, default_q)[0]),
        q_high=lambda d: d["pellet"].map(lambda x: quantiles_map.get(x, default_q)[1]),
    )
)
bounds = []
for _, row in q_by_pellet.iterrows():
    pel = row["pellet"]
    ql = row["q_low"]
    qh = row["q_high"]

    s = df_pos.loc[df_pos["pellet"] == pel, "performance"]
    bounds.append({
        "pellet": pel,
        "p_low": float(s.quantile(ql)),
        "p_high": float(s.quantile(qh)),
        "q_low": ql,
        "q_high": qh,
    })

bounds = pd.DataFrame(bounds).set_index("pellet")


prod_pel_with_bounds = df_base.merge(
    bounds[["p_low", "p_high"]],
    left_on="pellet",
    right_index=True,
    how="left")

cond_trimmed = (
    (prod_pel_with_bounds["performance"] > 0) &
    (prod_pel_with_bounds["performance"] >= prod_pel_with_bounds["p_low"]) &
    (prod_pel_with_bounds["performance"] <= prod_pel_with_bounds["p_high"])
)

prod_pel_trimmed = prod_pel_with_bounds[cond_trimmed].copy()
conds_bad = (~cond_trimmed) & (prod_pel_with_bounds["performance"] > 0)
prod_pel_bad = prod_pel_with_bounds[conds_bad].copy()
prod_pel_bad["lot"] = pd.to_numeric(prod_pel_bad["lot"], errors="coerce").astype("Int64")
prod_pel_bad = prod_pel_bad.round(2).sort_values("performance")

In [11]:

cls_bad_prod = [
    "date", "lot", "pellet", "feed_type",
    "quantity_tm", "hour", "performance", "temperature_c",
    "operators", "hum", "pdi",
]

rename_map_bad = {
    "date": "fecha",
    "lot": "lote",
    "pellet": "pellet",
    "feed_type": "tipo_alimento",
    "quantity_tm": "cantidad_tm",
    "hour": "hora",
    "performance": "rendimiento",
    "temperature_c": "temperatura_c",
    "operators": "operario",
    "hum": "humedad",
    "pdi": "pdi",
}

prod_pel_bad = prod_pel_bad[cls_bad_prod].rename(columns=rename_map_bad)
s3.save_dataframe(prod_pel_bad, "detalle_bad.csv")
prod_pel_bad

,fecha,lote,pellet,tipo_alimento,cantidad_tm,hora,rendimiento,temperatura_c,operario,humedad,pdi
319,2025-09-18,31406,pellet 3,BCC,1.0,1.40,0.71,75.30,Brayan Erazo,11.85,98.48
105,2025-08-15,31170,pellet 1,F1,2.0,2.25,0.89,64.60,Jose Benalcazar,9.86,99.62
321,2025-09-18,31408,pellet 2,PF02,1.0,1.08,0.92,60.10,Brayan Erazo,9.18,99.51
54,2025-08-08,31118,pellet 2,PF01,3.0,3.25,0.92,60.50,Brayan Erazo,8.82,99.07
570,2025-10-30,31676,pellet 3,E-2,13.0,14.00,0.93,81.80,Jose Cabezas,11.76,94.20
240,2025-09-03,31319,pellet 3,F-3,1.0,1.00,1.00,75.10,Johnny Tirira,10.73,99.44
242,2025-09-03,31321,pellet 1,AIP -PREMEX,5.0,4.00,1.25,74.50,Jose Benalcazar,10.64,99.16
421,2025-10-07,31518,pellet 1,E-1,5.0,3.75,1.33,79.55,Johnny Tirira,10.77,99.21
395,2025-10-01,31490,pellet 3,P6,1.0,0.75,1.33,77.90,Jose Cabezas,12.14,98.71
401,2025-10-02,31497,pellet 1,E-1,3.0,2.17,1.38,80.30,Johnny Tirira,10.62,98.82


In [12]:
bounds

,p_low,p_high,q_low,q_high
pellet,,,,
pellet 3,2.253559,7.500000,0.03,0.99
pellet 2,1.799862,10.482932,0.07,0.99
pellet 1,1.395677,7.174716,0.03,0.99


In [13]:
prod_pel_bad_group = (
    prod_pel_bad
    .groupby(["pellet"])
    .agg(
        lot=("lote", "count"),
        feed_type=("tipo_alimento", "unique"),
        performance_min=("rendimiento", "min"),
        performance_max=("rendimiento", "max"),
        performance_median=("rendimiento", "median"),
    )
    .reset_index()
)

prod_pel_bad_group = prod_pel_bad_group.rename(columns={
    "lot": "cantidad_lotes",
    "feed_type": "tipos_alimento",
    "performance_min": "rendimiento_min",
    "performance_max": "rendimiento_max",
    "performance_median": "rendimiento_mediana",
})



prod_pel_bad_group["tipos_alimento"] = prod_pel_bad_group["tipos_alimento"].apply(join_unique)

s3.save_dataframe(prod_pel_bad_group, "prod_pel_bad.csv")
prod_pel_bad_group

,pellet,cantidad_lotes,tipos_alimento,rendimiento_min,rendimiento_max,rendimiento_mediana
0,pellet 1,7,"AIP -PREMEX, BAI, E-1, F1, HY-8",0.89,8.78,1.380
1,pellet 2,18,"ACP, E-2, E-3, F1, F1 PREMEX, F2 PREMEX, PF01,...",0.92,15.48,1.710
2,pellet 3,8,"BAI, BCC, E-1, E-2, E-3, F-3, P6, PCR",0.71,12.00,1.355


In [14]:
performance_trimmed_group = prod_pel_trimmed.groupby(["pellet", "tiro"]).agg(
    lotes_q =("lot", "nunique"),
    rendimiento_q=('performance', 'mean'),
).reset_index().round(1)
performance_trimmed_group

,pellet,tiro,lotes_q,rendimiento_q
0,pellet 1,corto,114,2.9
1,pellet 1,largo,14,3.2
2,pellet 2,corto,119,4.0
3,pellet 2,largo,78,5.7
4,pellet 3,corto,156,4.4
5,pellet 3,largo,57,5.7


In [15]:

prod_pel_summary = prod_pel_group.groupby(["pellet"]).agg(
    cantidad_ops=("lot", "nunique"),
    toneladas_peletizadas=("quantity_tm", "sum"),
    pdi_promedio=('pdi', 'mean'),
    temperatura_promedio=("temperature_c", "mean"),
    humedad_promedio=('hum', "mean"),
).reset_index()

prod_pel_summary["pdi_promedio"] = prod_pel_summary["pdi_promedio"].astype(float)
prod_pel_summary = prod_pel_summary.round(2)
prod_pel_summary = pd.merge(prod_pel_summary, performance_trimmed_group, on="pellet")

rename_summary = {
    "pellet": "pellet",
    "cantidad_ops": "Cantidad Lotes",
    "toneladas_peletizadas": "Toneladas Peletizadas",
    "rendimiento_q": "Rendimiento (Ton/H) (P3/P98)",
    "pdi_promedio": "PDI (%)",
    "temperatura_promedio": "Temperatura (°C)",
    "humedad_promedio": "Humedad (%)",

             }
prod_pel_summary = prod_pel_summary.rename(
    columns=rename_summary)[rename_summary.values()]
s3.save_dataframe(prod_pel_bad_group, "summary.csv")
prod_pel_summary

,pellet,Cantidad Lotes,Toneladas Peletizadas,Rendimiento (Ton/H) (P3/P98),PDI (%),Temperatura (°C),Humedad (%)
0,pellet 1,157,1599.0,2.9,98.21,80.35,10.46
1,pellet 1,157,1599.0,3.2,98.21,80.35,10.46
2,pellet 2,276,3840.0,4.0,98.09,75.42,11.26
3,pellet 2,276,3840.0,5.7,98.09,75.42,11.26
4,pellet 3,294,3818.0,4.4,97.75,78.71,11.69
5,pellet 3,294,3818.0,5.7,97.75,78.71,11.69


In [16]:
prod_pel_group[prod_pel_group["pellet"] == 'pellet 1'].groupby(["specie", "month"]).agg(
    tons=("quantity_tm", "sum"),
    list=("feed_type","unique"))


tons                                            list
specie        month                                                       
broiler       4        3.0                                           [E-1]
              5       30.0                                           [E-1]
              8       30.0                                           [E-1]
              9       57.0                                           [E-1]
              10     133.0                                           [E-1]
              11      70.0                                           [E-1]
              12       6.0                                           [E-1]
cerdos        8       28.0                         [PF-01, PF-02, F1, C-1]
              9       30.0                                      [C-1, BCE]
cuyes         8        5.0                                           [C-2]
ganaderia     6       30.0                               [G2, G4, G5, G-1]
              8      131.0                             [G5PR, G-3, G5, G2]
              9      123.0  [G-2, G-4, G1, G-5PR K, G-3, G-5, G-1, G-5 PR]
              11      30.0                                           [G-3]
ponedora      8       50.0                                      [APP, BAI]
              9       28.0                              [AIP -PREMEX, APP]
              10     176.0                    [AIP -PREMEX, APP, BAI, AIP]
              11     161.0                                 [BAI, AIP, APP]
reproductoras 4       12.0                                    [HY-1, HY-8]
              8       47.0                       [HY-10, HY-8, HY-6, HY-1]
              9      148.0                 [HY-2, HY-10, HY-8, HY-6, HY-3]
              10     114.0                [HY-3, HY-7, HY-8, HY-11, HY-12]
              11     136.0                              [HY-3, HY-8, HY-4]
              12      21.0                        [HY-6, HY-7, HY-8, HY-1]

In [17]:

prod_pel_summary = prod_pel_group.groupby(["specie"]).agg(
    cantidad_ops=("lot", "nunique"),
    toneladas_peletizadas =("quantity_tm", "sum"),
    horas_proceso=('hour', "sum"),
    pdi_promedio=('pdi', 'mean'),
    rendimiento=("performance", "mean"),
    temperatura_promedio=("temperature_c", "mean"),
    horas_paro=("downtime_hour", "sum"),
    humedad_promedio=('hum', "mean"),
).reset_index()
prod_pel_summary["pdi_promedio"] = prod_pel_summary["pdi_promedio"].astype(float)
prod_pel_summary = prod_pel_summary.round(2)

prod_pel_summary.rename(
    columns={"pdi_promedio": "PDI (%)",
             "cantidad_ops": "Cantidad Lotes",
             "toneladas_peletizadas": "Toneladas Peletizadas",
             "horas_proceso": "Horas Proceso",
             "temperatura_promedio": "Temperatura (°C)",
             "rendimiento": "Rendimiento (Ton/H)",
             "humedad_promedio": "Humedad (%)",

             }, inplace=True)
s3.save_dataframe(prod_pel_summary, "summary_especie.csv")
prod_pel_summary

,specie,Cantidad Lotes,Toneladas Peletizadas,Horas Proceso,PDI (%),Rendimiento (Ton/H),Temperatura (°C),horas_paro,Humedad (%)
0,broiler,232,4651.0,712.45,97.48,5.03,80.51,28.63,11.68
1,cerdos,302,2827.0,588.28,98.40,4.01,75.30,22.02,11.28
2,cuyes,7,51.0,4.67,98.71,3.43,78.67,0.00,12.38
3,ganaderia,55,541.0,100.17,97.87,4.17,76.57,5.17,10.86
4,maquila,7,8.0,2.25,98.04,3.16,73.99,0.00,11.79
5,ponedora,54,667.0,231.58,98.70,2.82,78.05,6.42,11.03
6,reproductoras,62,512.0,147.18,97.18,2.88,81.18,3.87,9.97


In [18]:
group_pel_specie =prod_pel_trimmed.groupby(["pellet", "specie"]).agg(# prod_pel_group
    cantidad_ops=("lot", "nunique"),
    toneladas_peletizadas =("quantity_tm", "sum"),
    horas_proceso=('hour', "sum"),
    pdi_promedio=('pdi', 'mean'),
    rendimiento=("performance", "mean"),
    temperatura_promedio=("temperature_c", "mean"),
    horas_paro=("downtime_hour", "sum"),
    humedad_promedio=('hum', "mean"),
).reset_index()
group_pel_specie

,pellet,specie,cantidad_ops,toneladas_peletizadas,horas_proceso,pdi_promedio,rendimiento,temperatura_promedio,horas_paro,humedad_promedio
0,pellet 1,broiler,20,266.0,113.133333,98.971111,2.391276,82.484167,2.916667,10.870556
1,pellet 1,cerdos,6,56.0,17.500000,98.440000,3.178743,75.250000,0.000000,10.996667
2,pellet 1,cuyes,1,5.0,1.500000,98.840000,3.333333,77.200000,0.000000,11.760000
3,pellet 1,ganaderia,29,299.0,71.666667,97.783929,3.996414,77.881034,2.833333,10.703214
4,pellet 1,ponedora,19,276.0,125.000000,99.127895,2.286480,81.372222,1.500000,10.807368
5,pellet 1,reproductoras,53,406.0,141.600000,97.551154,2.759802,81.618239,3.866667,9.853077
6,pellet 2,broiler,97,2120.0,370.316667,97.461146,5.529819,79.497222,15.716667,11.735729
7,pellet 2,cerdos,83,700.0,175.416667,98.721728,3.956777,72.644444,6.583333,11.048072
8,pellet 2,cuyes,1,5.0,1.416667,99.030000,3.529412,74.100000,0.000000,12.280000
9,pellet 2,maquila,1,1.0,0.416667,98.580000,2.400000,78.200000,0.000000,11.200000


In [19]:

feed_type_pel23 = prod_pel_group[(prod_pel_group["pellet"].isin(['pellet 2', 'pellet 3'])) &(prod_pel_group["performance"]>0)].pivot_table(
        index='feed_type',
        columns='pellet',
        values='quantity_tm',
        observed=False

    ).dropna().reset_index()['feed_type']
cond_pels = prod_pel_group["pellet"].isin(['pellet 2', 'pellet 3'])
cond_feed =  prod_pel_group["feed_type"].isin(feed_type_pel23)

In [20]:
HUM_MIN = 11.5
HUM_MAX = 12.5

summary_hum = resumen_cumplimiento_rango(
    df=prod_pel_group,
    specie_col="specie",
    lot_col="lot",
    value_col="hum",
    target_min=HUM_MIN,
    target_max=HUM_MAX,
    unit="%"    # para mostrar en rango_objetivo
)

s3.save_dataframe(summary_hum, "summary_hum.csv")
summary_hum


,Especie,Rango objetivo,Valor promedio,Cantidad de lotes,Lotes en rango,Pct cumplimiento
6,reproductoras,(11.5 - 12.5) %,9.97,62,3,4.84
3,ganaderia,(11.5 - 12.5) %,10.86,55,15,27.27
5,ponedora,(11.5 - 12.5) %,11.03,54,17,31.48
1,cerdos,(11.5 - 12.5) %,11.27,302,117,38.74
0,broiler,(11.5 - 12.5) %,11.68,232,109,46.98
2,cuyes,(11.5 - 12.5) %,12.38,7,4,57.14
4,maquila,(11.5 - 12.5) %,11.79,7,4,57.14


In [21]:
dict_temp_species_max = {
    "cerdos": 85,
    "ganaderia": 85,
    "broiler": 85,
    "reproductoras": 85,
    "cuyes": 82,
    "maquila": 85,
    "ponedora": 85

}
summary_temp = resumen_cumplimiento_rango(
    df=prod_pel_group[prod_pel_group["performance"]>0],
    specie_col="specie",
    lot_col="lot",
    value_col="temperature_c",
    target_min=60,
    target_max=dict_temp_species_max,
    unit="°C"
)
s3.save_dataframe(summary_temp, "summary_temp.csv")
summary_temp

,Especie,Rango objetivo,Valor promedio,Cantidad de lotes,Lotes en rango,Pct cumplimiento
0,broiler,(60.0 - 85) °C,80.12,171,165,96.49
1,cerdos,(60.0 - 85) °C,75.47,245,237,96.73
5,ponedora,(60.0 - 85) °C,77.91,42,41,97.62
3,ganaderia,(60.0 - 85) °C,77.18,43,42,97.67
2,cuyes,(60.0 - 82) °C,77.20,3,3,100.00
4,maquila,(60.0 - 85) °C,74.78,5,5,100.00
6,reproductoras,(60.0 - 85) °C,81.56,55,55,100.00


In [22]:
# Ejemplo 1: Temperatura con conteo de lotes
RANGE_PDI = (90, 100)
fig = plot_gauge_grid(
    df=prod_pel_group,
    group_col="pellet",
    value_col="pdi",
    target_ranges=RANGE_PDI,
    count_col="lot",  # Cuenta lotes distintos
    label_count="Lotes",
    label_metric="PDI",
    order_groups=["pellet 1", "pellet 2", "pellet 3"],
    title_prefix="PDI",
    unit="%",
)
s3.save_plotly_html(fig, "gauge_pellet_pdi.html")

fig.show()


In [23]:
pdi_bad = prod_pel_group[prod_pel_group["pdi"]<RANGE_PDI[0]][cls_bad_prod].rename(columns=rename_map_bad).round(2)
s3.save_dataframe(pdi_bad, "pdi_bad.csv")
pdi_bad

,fecha,lote,pellet,tipo_alimento,cantidad_tm,hora,rendimiento,temperatura_c,operario,humedad,pdi
412,2025-10-06,31508,pellet 1,HY-11,8.0,2.67,3.0,76.2,Johnny Tirira,10.18,89.47


In [24]:
# Ejemplo 1: Temperatura con conteo de lotes
fig = plot_gauge_grid(
    df=prod_pel_trimmed,
    group_col="pellet",
    value_col="performance",
    target_ranges={
        "pellet 1": (2, 8),
        "pellet 2": (4, 10),
        "pellet 3": (4, 10),
    },
    order_groups=["pellet 1", "pellet 2", "pellet 3"],

    count_col="lot",  # Cuenta lotes distintos
    title_prefix="Rendimiento",
    unit="Ton/H",
)
s3.save_plotly_html(fig, "gauge_pellet_rend.html")
fig.show()


In [25]:


dietas_pellet = (
    prod_pel_trimmed
    .groupby(["pellet", "feed_type"], as_index=False)
    .agg(
        rendimiento=("performance", "mean"),
        quantity_tm=("quantity_tm", "sum"),
    )
)
targets = {
    "pellet 1": prod_pel_trimmed[prod_pel_trimmed["pellet"] == "pellet 1"]["performance"].mean(),
    "pellet 2":  prod_pel_trimmed[prod_pel_trimmed["pellet"] == "pellet 2"]["performance"].mean(),
    "pellet 3":  prod_pel_trimmed[prod_pel_trimmed["pellet"] == "pellet 3"]["performance"].mean(),
}

dietas_pellet["global_mean_ton_h"] = dietas_pellet["pellet"].map(targets)
cond_diet_bad_performance = dietas_pellet["rendimiento"] < dietas_pellet["global_mean_ton_h"]

min_tm = 30
cond_diet_min_ton =dietas_pellet["quantity_tm"] > min_tm
dietas_bad_performance = dietas_pellet[cond_diet_bad_performance & cond_diet_min_ton]
peores_5_por_pellet = (
    dietas_bad_performance.sort_values("rendimiento", ascending=True).groupby("pellet").head(5)
)
peores_5_por_pellet

,pellet,feed_type,rendimiento,quantity_tm,global_mean_ton_h
3,pellet 1,BAI,2.151645,80.0,2.936249
1,pellet 1,AIP -PREMEX,2.263878,79.0,2.936249
2,pellet 1,APP,2.306647,90.0,2.936249
33,pellet 2,AIP,2.384528,37.0,4.697864
7,pellet 1,E-1,2.391276,266.0,2.936249
25,pellet 1,HY-3,2.455672,40.0,2.936249
70,pellet 2,PF02,2.613913,77.0,4.697864
47,pellet 2,E-1,2.931303,88.0,4.697864
35,pellet 2,BAI,3.042754,44.0,4.697864
36,pellet 2,BCC,3.536055,48.0,4.697864


In [26]:

corporate_colors = [
        "#1A494C", "#17877D", "#94AF92", "#F6B27A", "#F18F01",
        "#E4572E", "#6C757D", "#343A40", "#A3CED0",
    ]
fig = plot_bar(
    df=peores_5_por_pellet.round(2),
    x_col="pellet",
    group_col="feed_type",
    y_col="rendimiento",
    hover_data_cols=["quantity_tm"],
    title="",
    x_title="Pellet",
    order_x=["pellet 1", "pellet 2", "pellet 3"],
    y_title="Rendimiento (Ton/H)",
    bar_colors=corporate_colors,
    compact_mode=True,
    bar_width_scale=0.7,
    cluster_width=0.5,
)
fig.show()
s3.save_plotly_html(fig, "rendimiento_pellet_especie_decil.html")



In [27]:
# Paleta corporativa
CORP_DEEP_TEAL = "#1A494C"
CORP_TEAL      = "#1C8074"
CORP_SAGE      = "#94AF92"
CORP_GRAY      = "#6B7280"
ACCENT_WARM    = "#F6B27A"

corporate_colorscale = [
    (0.00, CORP_DEEP_TEAL),
    (0.25, CORP_TEAL),
    (0.50, CORP_GRAY),      # neutral en el medio
    (0.75, CORP_SAGE),
    (1.00, ACCENT_WARM),    # contraste agradable
]
corporate_colorway = [CORP_DEEP_TEAL, CORP_TEAL, CORP_SAGE, CORP_GRAY, ACCENT_WARM]

fig = plot_heatmap(
    df=prod_pel_trimmed,

    # Ejes
    x_col="pellet",
    y_col="operators",

    # Métrica Principal (Color + Centro)
    value_col="performance",
    aggfunc="mean",
    value_unit="Ton/H",
    decimals_value=2,

    # Métrica Secundaria (Esquina)
    secondary_col="quantity_tm",
    secondary_aggfunc="sum",
    secondary_prefix="Tons: ",
    secondary_position="bottom", # La etiqueta pequeña va abajo
    show_secondary_labels=False,


    # Estética
    colorscale=corporate_colorway,
    center_font_size=16,
    secondary_font_size=10,
    title="Matriz de Rendimiento: Operario vs Pellet",
    transparent_bg=True,
)
fig.show()
s3.save_plotly_html(fig, "heatmap_rendimiento.html")


In [28]:
ton_by_op_pellet = prod_pel_group.groupby(["pellet", "operators"]).agg(
    ton_prom=("quantity_tm", "mean"), quantity_tm=("quantity_tm", "sum")).reset_index()
ton_by_op_pellet

,pellet,operators,ton_prom,quantity_tm
0,pellet 1,Brayan Erazo,8.000000,8.0
1,pellet 1,Johnny Tirira,9.961538,518.0
2,pellet 1,Jose Benalcazar,10.450000,1045.0
3,pellet 1,Jose Cabezas,7.000000,28.0
4,pellet 2,Brayan Erazo,14.302817,2031.0
5,pellet 2,Johnny Tirira,11.900000,119.0
6,pellet 2,Jose Benalcazar,13.000000,104.0
7,pellet 2,Jose Cabezas,13.672414,1586.0
8,pellet 3,Brayan Erazo,13.621622,2016.0
9,pellet 3,Johnny Tirira,16.111111,145.0


In [29]:

fig = plot_heatmap(
    df=prod_pel_group,
    # Ejes
    x_col="pellet",
    y_col="operators",
    # Métrica Principal (Color + Centro)
    value_col="quantity_tm",
    aggfunc="sum",
    value_unit="Ton",
    decimals_value=0,

    secondary_col= 'quantity_tm',
    secondary_aggfunc= "mean",
    # Estética
    colorscale=corporate_colorway,
    center_font_size=16,
    secondary_font_size=10,
    show_secondary_labels=False,
    title="Toneladas Totales producidas por — Operario × Pellet",
    transparent_bg=True,
)
fig.show()
s3.save_plotly_html(fig, "pellet_heatmap_ton_pel_op.html")

In [30]:
df23 = prod_pel_trimmed[prod_pel_trimmed["pellet"].isin(["pellet 2", "pellet 3"])].copy()

# si "tiro" no existe aún, asegúralo antes:
# df23["tiro"] = np.where(df23["quantity_tm"] <= 15, "corto", "largo")

dietas_ambas_pel_y_ambos_tiros = (
    df23.groupby("feed_type")[["pellet", "tiro"]]
       .apply(lambda g: g.drop_duplicates().shape[0])  # # de pares únicos (pellet, tiro)
       .loc[lambda s: s == 4]
       .index
       .tolist()
)

cond_pel12 = prod_pel_trimmed["pellet"].isin(["pellet 2", "pellet 3"])
cond_feed12 = prod_pel_trimmed["feed_type"].isin(dietas_ambas_pel_y_ambos_tiros)
prod_pel_trimmed12 = prod_pel_trimmed[cond_pel12 & cond_feed12]
prod_pel_trimmed12_corto = prod_pel_trimmed12[prod_pel_trimmed12["tiro"] == "corto"].copy()
prod_pel_trimmed12_largo = prod_pel_trimmed12[prod_pel_trimmed12["tiro"] != "corto"].copy()


In [31]:
fig_23 = plot_heatmap(
    df=prod_pel_trimmed12_corto,
    x_col="feed_type",
    y_col="pellet",
    value_col="performance",
    show_secondary_labels=False,
    title="Rendimiento (Ton/H) por — Dieta × Pellet en tirajes cortos",
    width=1100, height=350,
    colorscale=corporate_colorscale,
)
fig_23.show()
s3.save_plotly_html(fig_23, "pellet_heatmap_rendimiento_pellet.html")

/Users/juandavidrincon/Documents/hawking/core/viz.py:619: FutureWarning:

pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.



In [32]:
fig_23 = plot_heatmap(
    df=prod_pel_trimmed12_largo,
    x_col="feed_type",
    y_col="pellet",
    value_col="performance",
    show_secondary_labels=False,
    secondary_col='quantity_tm',
    secondary_aggfunc = "mean",
    title="Rendimiento (Ton/H) por — Dieta × Pellet en tirajes largos",
    width=1100, height=350,
    colorscale=corporate_colorscale,
)
fig_23.show()
s3.save_plotly_html(fig_23, "pellet_heatmap_rendimiento_pellet.csv")

In [33]:
corporate_scale = [(0.0, "#17877D"), (0.5, "#FFFFFF"), (1.0, "#F6B27A")]
fig = plot_time_heatmap(
    df=prod_pel[prod_pel["downtime_hour"] > 0],
    x_col="pellet",
    y_col="normalize_notes",
    value_col="downtime_hour",
    input_unit='hours',
    title="Tiempo promedio de paros por  — Tipo × Pellet",
    colorscale=corporate_scale,
    width=1000,
    height=400,
    #output_path=f"{ROOT_IMAGEN}/pellet_heatmap_notes_time.html"
)
fig.show()
s3.save_plotly_html(fig, "pellet_heatmap_notes_time.html")



In [34]:
hour_avg_pellet = prod_pel_group[prod_pel_group["hour"] > 0]\
    .groupby(["date", "pellet"]).agg(hour=("hour", "sum")).reset_index()

hour_avg_pellet = hour_avg_pellet.groupby(["pellet"])\
    .agg(hour=("hour", "mean")).reset_index()

downtime_avg_pellet = prod_pel_group[prod_pel_group["downtime_hour"] > 0]\
    .groupby(["pellet", "date"]).agg(
        downtime_hour=("downtime_hour", "sum")
    ).reset_index()

downtime_avg_pellet = downtime_avg_pellet.groupby(["pellet"])\
    .agg(downtime_hour=("downtime_hour", "mean")).reset_index()

avg_time_pellet = pd.merge(downtime_avg_pellet, hour_avg_pellet, on=["pellet"], how="left")
avg_time_pellet["rate"] = avg_time_pellet["downtime_hour"] / avg_time_pellet["hour"]
avg_time_pellet["productive_hour"] = (
    avg_time_pellet["hour"] - avg_time_pellet["downtime_hour"]
)
avg_time_pellet["downtime_pct"] = avg_time_pellet["rate"] * 100
avg_time_pellet["productive_pct"] = 100 - avg_time_pellet["downtime_pct"]

# 1. (Opcional) Renombrar columnas para que se vean bonitas en la leyenda del gráfico
avg_time_pellet_clean = avg_time_pellet.rename(columns={
    "productive_pct": "Productivo",
    "downtime_pct": "Improductivo"
})

df_long = avg_time_pellet_clean.melt(
    id_vars=["pellet"],
    value_vars=["Productivo", "Improductivo"],
    var_name="pct_time",
    value_name="value"
)
df_long

,pellet,pct_time,value
0,pellet 1,Productivo,91.819780
1,pellet 2,Productivo,84.955054
2,pellet 3,Productivo,89.228788
3,pellet 1,Improductivo,8.180220
4,pellet 2,Improductivo,15.044946
5,pellet 3,Improductivo,10.771212


In [35]:
fig = plot_bar(
    df=df_long,
    x_col="pellet",
    group_col="pct_time",
    title="Distribución del tiempo de operación por pellet (día promedio)",
    y_col="value",
    show_delta=False,
    order_x=["pellet 1", "pellet 2", "pellet 3"],
    y_title="Tiempo Operación (%)",
    #output_path=f"{ROOT_IMAGEN}/avg_time_pellet_stack.html",

)
fig.show()
s3.save_plotly_html(fig, "avg_time_pellet_stack.html")

In [36]:
df_downtime = prepare_pie_data(
    df=prod_pel[prod_pel["downtime_hour"] > 0],
    category_col="normalize_notes",
    value_col="downtime_hour",

    # Configurar el Hover específico para Tiempos
    include_sum=True,
    include_mean=True,
    include_count=True,

    labels={
        "sum": "Tiempo Total (h)",
        "mean": "Duración Promedio",
        "count": "Eventos"
    },
    formatters={
        "mean": _hours_to_hms_str,
        "sum": lambda x: f"{x:.1f} h"
    }
)
df_downtime

,label,count,sum,mean,slice_value,hover_html
7,Sin Producto,31,27.433333,0.884946,27.433333,<b>Sin Producto</b><br>Tiempo Total (h): 27.4 ...
0,Atasque,26,21.833333,0.839744,21.833333,<b>Atasque</b><br>Tiempo Total (h): 21.8 h<br>...
5,Falla del Sistema,7,5.800000,0.828571,5.800000,<b>Falla del Sistema</b><br>Tiempo Total (h): ...
8,Tolva Llena,10,5.583333,0.558333,5.583333,<b>Tolva Llena</b><br>Tiempo Total (h): 5.6 h<...
1,Cambio Dado/Rodillo,5,5.000000,1.000000,5.000000,<b>Cambio Dado/Rodillo</b><br>Tiempo Total (h)...
4,Falla Electrica,3,4.000000,1.333333,4.000000,<b>Falla Electrica</b><br>Tiempo Total (h): 4....
3,Falla Caldero,6,3.616667,0.602778,3.616667,<b>Falla Caldero</b><br>Tiempo Total (h): 3.6 ...
2,Experimental,2,2.583333,1.291667,2.583333,<b>Experimental</b><br>Tiempo Total (h): 2.6 h...
6,No especificado,1,0.333333,0.333333,0.333333,<b>No especificado</b><br>Tiempo Total (h): 0....


In [37]:
fig = plot_pie(
    df_prepared=df_downtime,
    title="<b>Distribución Causas de Paro (Por Tiempo)</b>",
    #output_path=f"{ROOT_IMAGEN}/downtime_pie_time.html"
)
fig.show()
s3.save_plotly_html(fig, "downtime_pie_time.html")


In [38]:
prod_pel_group_paros_red = prod_pel_group.groupby(["pellet", "with_notes"]).agg(
    cantidad_ops=("lot", "nunique"), #
    toneladas_peletizadas =("quantity_tm", "sum"),
    horas_proceso=('hour', "sum"),
    temperatura_promedio=("temperature_c", "mean"),
    horas_paro=("downtime_hour", "mean"),
    humedad_promedio=('hum', "mean"),
    pdi_promedio=('pdi', 'mean'),
     rendimiento=("performance", "mean"), #
).reset_index()
prod_pel_group_paros_red

,pellet,with_notes,cantidad_ops,toneladas_peletizadas,horas_proceso,temperatura_promedio,horas_paro,humedad_promedio,pdi_promedio,rendimiento
0,pellet 1,con paro,24,368.0,122.483333,80.369444,0.595139,10.449130,98.544348,3.211147
1,pellet 1,sin paro,133,1231.0,422.500000,80.346565,0.002506,10.462154,98.147846,2.725383
2,pellet 2,con paro,27,678.0,107.550000,79.379630,1.041975,11.597037,97.826154,5.760373
3,pellet 2,sin paro,249,3162.0,546.933333,74.974201,0.000000,11.227004,98.122857,4.328485
4,pellet 3,con paro,29,526.0,88.366667,79.640476,0.805172,11.567931,98.105172,5.195094
5,pellet 3,sin paro,265,3292.0,498.750000,78.603958,0.000000,11.704411,97.712481,4.544887


In [39]:
orden_formula = ["pellet 1", "pellet 2", "pellet 3"]  # ajusta al orden que quieras

fig = plot_bar(
    prod_pel_group_paros_red,
    title="Rendimiento promedio en lotes peletizados con y sin paros por pellet",
    x_col="pellet",
    cat_base = "sin paro",
    group_col="with_notes",
    y_col="rendimiento",
    show_delta=True,
    order_x=["pellet 1", "pellet 2", "pellet 3"],
    y_title="Rendimiento (Ton/H)",
    #output_path=f"{ROOT_IMAGEN}/rendimiento_lotes_with_without_downtime.html",
)
fig.show()
s3.save_plotly_html(fig, "rendimiento_lotes_with_without_downtime.html")